In [1]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Load CSV into dataframe
csv_file = "vaccinationsdata.csv"
vaccination_df = pd.read_csv(csv_file)
vaccination_df.head()

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,...,Administered_Dose1_Recip_5PlusPop_Pct,Series_Complete_5Plus,Series_Complete_5PlusPop_Pct,Administered_5Plus,Admin_Per_100k_5Plus,Distributed_Per_100k_5Plus,Series_Complete_Moderna_5Plus,Series_Complete_Pfizer_5Plus,Series_Complete_Janssen_5Plus,Series_Complete_Unk_Manuf_5Plus
0,01/21/2022,3,LA,7467120,318200,2957620,4191300,0,160625,190327,...,63.2,2390208.0,55.0,5865724.0,134927.0,171764.0,939747.0,1274979.0,174438.0,1044.0
1,01/21/2022,3,WA,16011165,734100,5776120,9500945,0,210261,246291,...,83.1,5298973.0,74.0,13190008.0,184259.0,223669.0,1853801.0,2999674.0,442148.0,3350.0
2,01/21/2022,3,CO,11371385,481900,4205300,6684185,0,197463,230374,...,81.6,3904471.0,72.0,9921557.0,182834.0,209551.0,1404341.0,2192272.0,305529.0,2329.0
3,01/21/2022,3,NV,5402390,255200,1851080,3296110,0,175393,206056,...,76.9,1786275.0,61.7,4525049.0,156328.0,186638.0,589391.0,1027647.0,169180.0,57.0
4,01/21/2022,3,HI,3148920,123800,1160860,1864260,0,222401,260003,...,89.7,1068153.0,80.3,2666120.0,200362.0,236645.0,357326.0,647324.0,63486.0,17.0


In [3]:
#Clean dataframe
clean_df = vaccination_df[['Date', 'Location', 'Administered']]
clean_df.head()

,Date,Location,Administered
0,01/21/2022,LA,5866023
1,01/21/2022,WA,13190985
2,01/21/2022,CO,9922591
3,01/21/2022,NV,4525127
4,01/21/2022,HI,2666683


In [4]:
clean_df = clean_df.rename(columns={'Location': 'State'})
clean_df.head()

,Date,State,Administered
0,01/21/2022,LA,5866023
1,01/21/2022,WA,13190985
2,01/21/2022,CO,9922591
3,01/21/2022,NV,4525127
4,01/21/2022,HI,2666683


In [5]:
aggregate_df = clean_df.loc[(clean_df['State'] == 'CA') |
             (clean_df['State'] == 'NV') |
             (clean_df['State'] == 'UT') |
             (clean_df['State'] == 'AZ')]
aggregate_df.head()

,Date,State,Administered
3,01/21/2022,NV,4525127
5,01/21/2022,CA,69706872
42,01/21/2022,UT,4699376
53,01/21/2022,AZ,10994834
79,01/20/2022,CA,69549938


In [6]:
#Split into regions
aggregate_df.loc[(aggregate_df['State'] == 'CA') |
             (aggregate_df['State'] == 'NV') |
             (aggregate_df['State'] == 'UT') |
             (aggregate_df['State'] == 'AZ'), ['State']] = 'SW'

C:\Users\Ryan\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [7]:
clean_aggregate_df = aggregate_df.rename(columns={'State': 'Region'})
clean_aggregate_df.head()

,Date,Region,Administered
3,01/21/2022,SW,4525127
5,01/21/2022,SW,69706872
42,01/21/2022,SW,4699376
53,01/21/2022,SW,10994834
79,01/20/2022,SW,69549938


In [8]:
#Change Date
clean_aggregate_df['Date'] = pd.to_datetime(clean_aggregate_df['Date'])
clean_aggregate_df.sort_values(by='Date')

,Date,Region,Administered
26125,2020-12-14,SW,0
26079,2020-12-14,SW,0
26118,2020-12-14,SW,0
26082,2020-12-14,SW,0
26013,2020-12-15,SW,0
...,...,...,...
79,2022-01-20,SW,69549938
53,2022-01-21,SW,10994834
42,2022-01-21,SW,4699376
5,2022-01-21,SW,69706872


In [9]:
#Sort by Date
final_df = clean_aggregate_df.groupby('Date').sum()
final_df.sort_values(by='Date')
final_df.head(10)

,Administered
Date,
2020-12-14,0
2020-12-15,0
2020-12-16,177
2020-12-17,1051
2020-12-18,2246
2020-12-19,19407
2020-12-20,59870
2020-12-21,91531
2020-12-22,116707
